In [88]:
%run ./config.py
import vectorbtpro as vbt 
import pandas as pd
import numpy as np

In [89]:
data = vbt.BinanceData.fetch('LUNAUSDT')
data

0it [00:00, ?it/s]

In [90]:
data.data['LUNAUSDT'].vbt.ohlcv.plot()

FigureWidget({
    'data': [{'close': array([ 0.4314,  0.4712,  0.4821, ..., 82.35  , 77.3   , 73.55  ]),
    …

In [91]:
open_price = data.get('Open')
close_price = data.get('Close')

In [92]:
vbt.RSI

vectorbtpro.indicators.custom.RSI

In [93]:
rsi = vbt.RSI.run(open_price)
rsi

In [94]:
rsi.rsi

Open time
2020-08-21 00:00:00+00:00          NaN
2020-08-22 00:00:00+00:00          NaN
2020-08-23 00:00:00+00:00          NaN
2020-08-24 00:00:00+00:00          NaN
2020-08-25 00:00:00+00:00          NaN
                               ...    
2022-05-03 00:00:00+00:00    43.170927
2022-05-04 00:00:00+00:00    36.346397
2022-05-05 00:00:00+00:00    41.494845
2022-05-06 00:00:00+00:00    41.593800
2022-05-07 00:00:00+00:00    34.593465
Freq: D, Name: Open, Length: 625, dtype: float64

In [95]:
entries = rsi.rsi.vbt.crossed_below(45)
entries

Open time
2020-08-21 00:00:00+00:00    False
2020-08-22 00:00:00+00:00    False
2020-08-23 00:00:00+00:00    False
2020-08-24 00:00:00+00:00    False
2020-08-25 00:00:00+00:00    False
                             ...  
2022-05-03 00:00:00+00:00     True
2022-05-04 00:00:00+00:00    False
2022-05-05 00:00:00+00:00    False
2022-05-06 00:00:00+00:00    False
2022-05-07 00:00:00+00:00    False
Freq: D, Name: Open, Length: 625, dtype: bool

In [96]:
exits = rsi.rsi.vbt.crossed_above(80)
exits

Open time
2020-08-21 00:00:00+00:00    False
2020-08-22 00:00:00+00:00    False
2020-08-23 00:00:00+00:00    False
2020-08-24 00:00:00+00:00    False
2020-08-25 00:00:00+00:00    False
                             ...  
2022-05-03 00:00:00+00:00    False
2022-05-04 00:00:00+00:00    False
2022-05-05 00:00:00+00:00    False
2022-05-06 00:00:00+00:00    False
2022-05-07 00:00:00+00:00    False
Freq: D, Name: Open, Length: 625, dtype: bool

In [97]:
def plot_rsi(rsi, entries, exits):
    fig = rsi.plot()
    entries.vbt.signals.plot_as_entry_markers(rsi.rsi, fig=fig)
    exits.vbt.signals.plot_as_exit_markers(rsi.rsi, fig=fig)
    return fig

plot_rsi(rsi, entries, exits)

FigureWidget({
    'data': [{'name': 'RSI',
              'showlegend': True,
              'type': 'scatter',…

In [98]:
clean_entries, clean_exits = entries.vbt.signals.clean(exits)
plot_rsi(rsi, clean_entries, clean_exits)

FigureWidget({
    'data': [{'name': 'RSI',
              'showlegend': True,
              'type': 'scatter',…

In [99]:
pf = vbt.Portfolio.from_signals(
    close=close_price,
    entries=clean_entries,
    exits=clean_exits,
    size=100,
    size_type='value',
    init_cash='auto'
)
pf

In [100]:
pf.stats()

Start                         2020-08-21 00:00:00+00:00
End                           2022-05-07 00:00:00+00:00
Period                                625 days 00:00:00
Start Value                                       100.0
End Value                                    442.840596
Total Return [%]                             342.840596
Benchmark Return [%]                       16949.142327
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              36.788411
Max Drawdown Duration                 171 days 00:00:00
Total Trades                                          6
Total Closed Trades                                   5
Total Open Trades                                     1
Open Trade PnL                               -18.630379
Win Rate [%]                                       60.0
Best Trade [%]                               295.786963
Worst Trade [%]                              -38

In [101]:
pf.plot(settings=dict(bm_returns=False))

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'na…

## Multiple backtests

In [102]:
def test_rsi(window=14, ewm=False, lower_th=30, upper_th=70):
    rsi = vbt.RSI.run(open_price, window=window, ewm=ewm)
    entries = rsi.rsi_crossed_below(lower_th)
    exits = rsi.rsi_crossed_above(upper_th)
    pf = vbt.Portfolio.from_signals(
        close=close_price,
        entries=entries,
        exits=exits, 
        size=100,
        size_type='value',
        init_cash='auto')
    return pf.stats([
        'total_return',
        'total_trades',
        'win_rate',
        'expectancy'
    ])

test_rsi()

Total Return [%]     71.25359
Total Trades                6
Win Rate [%]             80.0
Expectancy          16.564779
dtype: object

In [103]:
test_rsi(lower_th=20, upper_th=80)

Total Return [%]    -3.993737
Total Trades                3
Win Rate [%]        33.333333
Expectancy          -1.351246
dtype: object

In [104]:
from itertools import product

In [105]:
lower_ths = range(20, 31)
upper_ths = range(70, 81)
th_combs = list(product(lower_ths, upper_ths))
len(th_combs)

121

In [106]:
comb_stats = [
    test_rsi(lower_th=lower_th, upper_th=upper_th)
    for lower_th, upper_th in th_combs
]

In [107]:
comb_stats_df = pd.DataFrame(comb_stats)
comb_stats_df

,Total Return [%],Total Trades,Win Rate [%],Expectancy
0,25.013270,3,100.000000,8.337757
1,25.013270,3,100.000000,8.337757
2,25.013270,3,100.000000,8.337757
3,-25.995679,3,66.666667,-12.284551
4,-25.995679,3,66.666667,-12.284551
...,...,...,...,...
116,29.963134,5,75.000000,11.566600
117,52.108858,5,75.000000,16.613994
118,90.046711,5,75.000000,23.690587
119,166.256637,5,75.000000,42.743069


In [108]:
comb_stats_df.index = pd.MultiIndex.from_tuples(
    th_combs, 
    names=['lower_th', 'upper_th']
)
comb_stats_df

Total Return [%]  Total Trades  Win Rate [%]  Expectancy
lower_th upper_th                                                          
20       70               25.013270             3    100.000000    8.337757
         71               25.013270             3    100.000000    8.337757
         72               25.013270             3    100.000000    8.337757
         73              -25.995679             3     66.666667  -12.284551
         74              -25.995679             3     66.666667  -12.284551
...                             ...           ...           ...         ...
30       76               29.963134             5     75.000000   11.566600
         77               52.108858             5     75.000000   16.613994
         78               90.046711             5     75.000000   23.690587
         79              166.256637             5     75.000000   42.743069
         80              169.624444             5     75.000000   43.585020

[121 rows x 4 columns]

In [109]:
comb_stats_df['Expectancy'].vbt.heatmap()

FigureWidget({
    'data': [{'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
               …

In [110]:
windows = list(range(8, 21))
ewms = [False, True]

In [111]:
rsi = vbt.RSI.run(
    open_price,
    window=windows,
    ewm=ewms,
    param_product=True)
rsi.rsi.columns

MultiIndex([( 8, False),
            ( 8,  True),
            ( 9, False),
            ( 9,  True),
            (10, False),
            (10,  True),
            (11, False),
            (11,  True),
            (12, False),
            (12,  True),
            (13, False),
            (13,  True),
            (14, False),
            (14,  True),
            (15, False),
            (15,  True),
            (16, False),
            (16,  True),
            (17, False),
            (17,  True),
            (18, False),
            (18,  True),
            (19, False),
            (19,  True),
            (20, False),
            (20,  True)],
           names=['rsi_window', 'rsi_ewm'])

In [112]:
lower_ths_prod, upper_ths_prod = zip(*product(lower_ths, upper_ths))

In [113]:
len(lower_ths_prod)

121

In [114]:
len(upper_ths_prod)

121

In [115]:
lower_th_index = pd.Index(lower_ths_prod, name = 'lower_th')
entries = rsi.rsi_crossed_below(lower_th_index)
entries.columns

MultiIndex([(20,  8, False),
            (20,  8,  True),
            (20,  9, False),
            (20,  9,  True),
            (20, 10, False),
            (20, 10,  True),
            (20, 11, False),
            (20, 11,  True),
            (20, 12, False),
            (20, 12,  True),
            ...
            (30, 16, False),
            (30, 16,  True),
            (30, 17, False),
            (30, 17,  True),
            (30, 18, False),
            (30, 18,  True),
            (30, 19, False),
            (30, 19,  True),
            (30, 20, False),
            (30, 20,  True)],
           names=['lower_th', 'rsi_window', 'rsi_ewm'], length=3146)

In [116]:
upper_th_index = pd.Index(upper_ths_prod, name='upper_th')
exits = rsi.rsi_crossed_above(upper_th_index)
exits.columns

MultiIndex([(70,  8, False),
            (70,  8,  True),
            (70,  9, False),
            (70,  9,  True),
            (70, 10, False),
            (70, 10,  True),
            (70, 11, False),
            (70, 11,  True),
            (70, 12, False),
            (70, 12,  True),
            ...
            (80, 16, False),
            (80, 16,  True),
            (80, 17, False),
            (80, 17,  True),
            (80, 18, False),
            (80, 18,  True),
            (80, 19, False),
            (80, 19,  True),
            (80, 20, False),
            (80, 20,  True)],
           names=['upper_th', 'rsi_window', 'rsi_ewm'], length=3146)

In [117]:
pf = vbt.Portfolio.from_signals(
    close=close_price,
    entries=entries,
    exits=exits,
    size=100,
    size_type='value',
    init_cash='auto'
)
pf

In [118]:
stats_df = pf.stats([
    'total_return',
    'total_trades',
    'win_rate',
    'expectancy'
], agg_func=None)
stats_df

Total Return [%]  Total Trades  \
lower_th upper_th rsi_window rsi_ewm                                   
20       70       8          False          209.058672            10   
                             True            69.176727            10   
                  9          False          147.541091             9   
                             True             1.383751             7   
                  10         False          170.047061             8   
...                                                ...           ...   
30       80       18         True            10.278271             3   
                  19         False          202.701389             3   
                             True            94.849964             3   
                  20         False           56.123726             3   
                             True            93.274015             3   

                                      Win Rate [%]  Expectancy  
lower_th upper_th rsi_window rsi_ewm                            
20       70       8          False      100.000000   20.905867  
                             True        66.666667   10.808664  
                  9          False       87.500000   20.547872  
                             True        50.000000    2.515171  
                  10         False      100.000000   26.627352  
...                                            ...         ...  
30       80       18         True        33.333333    3.426090  
                  19         False       50.000000   79.468364  
                             True       100.000000   24.444232  
                  20         False       50.000000   11.633034  
                             True       100.000000   24.444232  

[3146 rows x 4 columns]

In [119]:
print(pf.getsize())

1.7 MB


In [120]:
stats_df['Expectancy'].groupby('rsi_window').mean()


rsi_window
8     16.055972
9     22.466409
10    22.081139
11    18.173139
12     9.396657
13    14.595109
14    15.734080
15    17.772601
16    10.038102
17    21.494165
18    33.053151
19    42.212036
20    33.901230
Name: Expectancy, dtype: float64

In [121]:
stats_df.sort_values(by='Total Trades', ascending=False).head()

Total Return [%]  Total Trades  \
lower_th upper_th rsi_window rsi_ewm                                   
29       70       8          False          149.600665            15   
                             True           119.730183            15   
         71       8          True           119.730183            15   
30       71       8          True            87.521117            15   
         70       8          False          121.902980            15   

                                      Win Rate [%]  Expectancy  
lower_th upper_th rsi_window rsi_ewm                            
29       70       8          False       78.571429   14.289251  
                             True        78.571429   12.653711  
         71       8          True        78.571429   12.653711  
30       71       8          True        78.571429   11.144653  
         70       8          False       71.428571   11.783276

In [122]:
pf[(29, 71, 8, True)].plot_value()

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width':…

In [132]:
data = vbt.BinanceData.fetch(['BTCUSDT', 'ETHUSDT'])
data['Close'].get().vbt.rebase(100).vbt.plot()

  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

FigureWidget({
    'data': [{'name': 'BTCUSDT',
              'showlegend': True,
              'type': 'scatt…

KeyError: 'Level symbol not found'